In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

In [ ]:
# Function to apply landmark-based augmentation
def apply_landmark_augmentation(landmarks, angle_range=(-10, 10), scale_range=(0.9, 1.1)):
    augmented_landmarks = landmarks.copy()
    num_landmarks = landmarks.shape[0]

    # Apply augmentation to each landmark
    for i in range(num_landmarks):
        angle = np.random.uniform(angle_range[0], angle_range[1])
        rotation_matrix = np.array([[np.cos(np.radians(angle)), -np.sin(np.radians(angle))],
                                    [np.sin(np.radians(angle)), np.cos(np.radians(angle))]])
        augmented_landmarks[i, :2] = np.dot(augmented_landmarks[i, :2], rotation_matrix.T)
        scale_factor = np.random.uniform(scale_range[0], scale_range[1])
        augmented_landmarks[i, :2] *= scale_factor

    return augmented_landmarks.flatten()

In [ ]:
df = pd.read_csv('8taijiquan.csv')


In [ ]:
df.head()


In [ ]:
# Get unique values in the 'class' column
unique_classes = df['class'].unique()

# Display the unique values
print("Unique Classes:", unique_classes)

In [ ]:
# Check the distribution of classes
print(df['class'].value_counts())

In [ ]:
df[df['class']=='Horse Stance']


In [ ]:
# Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(df.drop('class', axis=1), df['class'], test_size=0.3, random_state=1234, stratify=df['class'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1234, stratify=y_temp)


In [ ]:
# Apply landmark-based augmentation to X_train
X_train_augmented = []
for index, row in X_train.iterrows():
    landmarks = np.array(row).reshape(-1, 4)  # Assuming each row represents landmarks for one sample
    augmented_landmarks = apply_landmark_augmentation(landmarks)
    X_train_augmented.append(augmented_landmarks)

X_train_augmented = pd.DataFrame(X_train_augmented, columns=X_train.columns)


In [ ]:
# Encode class labels to numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
# Define the FNN model with modifications
def create_fnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dropout(0.7))
    model.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dropout(0.7))
    model.add(layers.Dense(len(label_encoder.classes_), activation='softmax'))
    optimizer = Adam(learning_rate=0.0001)  # Adjust the learning rate as needed
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the FNN model with early stopping
input_shape_fnn = (X_train_augmented.shape[1],)
fnn_model = create_fnn_model(input_shape_fnn)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

fnn_history = fnn_model.fit(X_train_augmented, y_train_encoded, epochs=100, batch_size=32,
                             validation_data=(X_val, y_val_encoded), callbacks=[early_stopping])


In [ ]:
plt.plot(fnn_history.history['accuracy'], label='Training Accuracy')
plt.plot(fnn_history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# Evaluate the FNN model on the test set
test_loss_fnn, test_accuracy_fnn = fnn_model.evaluate(X_test, y_test_encoded)
print(f'Test Accuracy (FNN): {test_accuracy_fnn}')


In [ ]:
# Confusion Matrix for FNN
y_pred_fnn = fnn_model.predict(X_test)
y_pred_classes_fnn = np.argmax(y_pred_fnn, axis=1)
conf_matrix_fnn = confusion_matrix(y_test_encoded, y_pred_classes_fnn)
sns.heatmap(conf_matrix_fnn, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (FNN)')
plt.show()

# Classification Report for FNN
class_report_fnn = classification_report(y_test_encoded, y_pred_classes_fnn)
print('Classification Report (FNN):')
print(class_report_fnn)


In [ ]:
# K-Fold Cross-Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
cv_scores = []

for train_index, test_index in kf.split(df.drop('class', axis=1), df['class']):
    X_train_cv, X_test_cv = df.drop('class', axis=1).iloc[train_index], df.drop('class', axis=1).iloc[test_index]
    y_train_cv, y_test_cv = df['class'].iloc[train_index], df['class'].iloc[test_index]

    model_cv = create_fnn_model((X_train_cv.shape[1],))
    model_cv.fit(X_train_cv, label_encoder.transform(y_train_cv), epochs=50, batch_size=32,
                 validation_split=0.2, callbacks=[early_stopping], verbose=0)

    _, accuracy_cv = model_cv.evaluate(X_test_cv, label_encoder.transform(y_test_cv))
    cv_scores.append(accuracy_cv)

print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))

In [ ]:
# Convert the model to TensorFlow Lite format (optional)
converter = tf.lite.TFLiteConverter.from_keras_model(fnn_model)
tflite_model = converter.convert()
with open('model_fnn.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import tensorflow as tf

# Load the FNN model
saved_model_path_fnn = './saved_model_fnn'
loaded_fnn_model = tf.keras.models.load_model(saved_model_path_fnn)

# Load the TensorFlow Lite model
tflite_model_path = './model_rnn.tflite'  # Adjust the path accordingly
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize BlazePose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Start capturing video from the camera
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1200)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 800)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with BlazePose
    results = pose.process(frame_rgb)

    # Recolor image back to BGR for rendering
    frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

    # Detect Taijiquan Stances
    if results.pose_landmarks:
        # Extract Pose landmarks
        pose_landmarks = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks]).flatten())

        # Make Detections
        X = pd.DataFrame([pose_row])

        # Convert X to numpy array
        input_data = X.to_numpy().astype(np.float32)

        # Set the input tensor
        interpreter.set_tensor(input_details[0]['index'], input_data)

        # Run inference
        interpreter.invoke()

        # Get the output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])

        # Get the predicted class and probabilities
        body_language_class = np.argmax(output_data)
        body_language_prob = output_data[0]

        print(body_language_class, body_language_prob)

        # Display Probability
        cv2.putText(frame, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, str(round(body_language_prob[body_language_class], 2)),
                    (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Display detected class
        cv2.putText(frame, f'CLASS: {body_language_class}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)

        # Draw pose landmarks
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

    cv2.imshow('Pose Detection with ML', frame)

    # Check for exit key
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()